In [52]:
from datetime import datetime

class Comment(object):
    def __init__(self, email, content, created=None):
        self.email = email
        self.content = content
        self.created = created or datetime.now()

In [53]:
comment = Comment(email='zenuine1@gmail.com', content='foo bar')

In [54]:
from rest_framework import serializers

class CommentSerializer(serializers.Serializer):
    email = serializers.EmailField()
    content = serializers.CharField(max_length=200)
    created = serializers.DateTimeField()

In [55]:
serializer = CommentSerializer(comment)
serializer.data

{'email': 'zenuine1@gmail.com', 'content': 'foo bar', 'created': '2018-07-27T10:59:08.796731+09:00'}

In [56]:
from rest_framework.renderers import JSONRenderer

json = JSONRenderer().render(serializer.data)
json

b'{"email":"zenuine1@gmail.com","content":"foo bar","created":"2018-07-27T10:59:08.796731+09:00"}'

In [57]:
from django.utils.six import BytesIO
from rest_framework.parsers import JSONParser

stream = BytesIO(json)
data = JSONParser().parse(stream)

In [58]:
serializer = CommentSerializer(data=data)
serializer.is_valid()

True

In [59]:
serializer.validated_data

OrderedDict([('email', 'zenuine1@gmail.com'),
             ('content', 'foo bar'),
             ('created',
              datetime.datetime(2018, 7, 27, 10, 59, 8, 796731, tzinfo=<DstTzInfo 'Asia/Seoul' KST+9:00:00 STD>))])

In [60]:
class CommentSerializer(serializers.Serializer):
    email = serializers.EmailField()
    content = serializers.CharField(max_length=200)
    created = serializers.DateTimeField()

    def create(self, validated_data):
        return Comment(**validated_data)

    def update(self, instance, validated_data):
        instance.email = validated_data.get('email', instance.email)
        instance.content = validated_data.get('content', instance.content)
        instance.created = validated_data.get('created', instance.created)
        return instance

In [61]:
comment = serializer.save()

NotImplementedError: `create()` must be implemented.

In [62]:
serializer = CommentSerializer(data=data)

In [63]:
serializer = CommentSerializer(comment, data=data)

In [64]:
serializer.save(owner=request.user)

NameError: name 'request' is not defined

In [65]:
class ContactForm(serializers.Serializer):
    email = serializers.EmailField()
    message = serializers.CharField()

    def save(self):
        email = self.validated_data['email']
        message = self.validated_data['message']
        send_email(from=email, message=message)

SyntaxError: invalid syntax (<ipython-input-65-d7dccb8db819>, line 8)

In [66]:
serializer = CommentSerializer(data={'email': 'foobar', 'content': 'baz'})
serializer.is_valid()

AppRegistryNotReady: The translation infrastructure cannot be initialized before the apps registry is ready. Check that you don't make non-lazy gettext calls at import time.

In [67]:
serializer.errors

AssertionError: You must call `.is_valid()` before accessing `.errors`.

In [68]:
serializer.is_valid(raise_exception=True)

AppRegistryNotReady: The translation infrastructure cannot be initialized before the apps registry is ready. Check that you don't make non-lazy gettext calls at import time.

In [69]:
from rest_framework import serializers

class BlogPostSerializer(serializers.Serializer):
    title = serializers.CharField(max_length=100)
    content = serializers.CharField()
    
    def validate_title(self, value):
        """
        Check that the blog post is about Django.
        """
        if 'django' not in value.lower():
            raise serializers.ValidationError("Blog post is not about Django")
        return value

In [70]:
from rest_framework import serializers

class EventSerializer(serializers.Serializer):
    description = serializers.CharField(max_length=100)
    start = serializers.DateTimeField()
    finish = serializers.DateTimeField()
    
    def validate(self, data):
        if data['start'] > data['finish']:
            raise serializers.ValidationError("finish must occur after start")
        return data

In [71]:
def multiple_of_ten(value):
    if value % 10 != 0:
        raise serializers.ValidationError('Not a multiple of ten')
        
class GameRecord(serializers.Serializer):
    score = IntegerField(validators=[multiple_of_ten])
    ...

NameError: name 'IntegerField' is not defined

In [72]:
class EventSerializer(serializers.Serializer):
    name = serializers.CharField()
    room_number = serializers.IntegerField(choices=[101, 102, 103, 201])
    date = serializers.DateField()
    
    class Meta:
        # Each room only has one event per day.
        validators = UniqueTogetherValidator(
            queryset=Event.objects.all()
            fields=['room_number', 'date']
        )

SyntaxError: invalid syntax (<ipython-input-72-b5c721c7faa9>, line 10)

In [73]:
# Update 'comment' with partial data
serializer = CommentSerializer(comment , data={'content': u'foo bar'}, partial=True)


In [75]:
class UserSerializer(serializers.Serializer):
    email = serializers.EmailField()
    username = serializers.CharField(max_length=100)
    
class CommentSerializer(serializers.Serializer):
    user = UserSerializer(required=False)
    edits = EditItemSerializer(many=True)
    content = serializers.CharField(max_length=200)
    created = serializers.DateTimeField()

In [77]:
serializer = CommentSerializer(data={'user': {'email': 'foobar', 'username': 'doe'}, 'content': 'baz'})
serializer.is_valid()

AppRegistryNotReady: The translation infrastructure cannot be initialized before the apps registry is ready. Check that you don't make non-lazy gettext calls at import time.

In [78]:
serializer.errors

AssertionError: You must call `.is_valid()` before accessing `.errors`.

In [79]:
class UserSerializer(serializer.ModelSerializer):
    profile = ProfileSerializer()
    
    class Meta:
        model = User
        fields = ('username', 'email', 'profile')
        
    def create(self, validated_data):
        profile_data = validated_data.pop('profile')
        user = User.objects.create(**validated_data)
        Profile.objects.create(user=user, **profile_data)
        return user

AttributeError: 'CommentSerializer' object has no attribute 'ModelSerializer'

In [80]:
def update(self, instance, validated_data):
    profile_data = validated_data.pop('profile')
    profile = instance.profile
    
    instance.username = validated_data.get('username', instance.username)
    instance.email = validated_data.get('email', instance.email)
    instance.save()
    
    profile.is_premium_member = profile_data.get(
        'is_premium_member',
        profile.is_premium_member
    )
    profile.has_support_contract = profile_data.get(
        'has_support_contract',
        profile.has_support_contract
    )
    profile.save()
    
    return instance

In [81]:
class UserManager(models.Manager):
    ...
    
    def create(self, username, email, is_premium_members=False, has_support_contract=False):
        user = User(username=username, email=email)
        user.save()
        profile = Profile(
            user=user,
            is_premium_member=is_premium_member,
            has_support_contract=has_support_contract,
        )
        profile.save()
        return user

NameError: name 'models' is not defined

In [82]:
def create(self, validated_data):
    return User.objects.create(
        username=validated_data['username'],
        email=validated_data['email'],
        is_premium_member=validated_data['profile']['is_premium_member'],
        has_support_contract=validated_data['profile']['has_support_contract']
    )

SyntaxError: unexpected EOF while parsing (<ipython-input-82-db821390df7a>, line 1)

In [83]:
queryset = Book.objects.all()
serializer = BookSerializer(queryset, many=True)
serializer.data

NameError: name 'Book' is not defined

In [84]:
serializer = AccountSerializer(account, context={'request': request})
serializer.data

NameError: name 'AccountSerializer' is not defined

In [85]:
class AccountSerializer(serializers.ModelSerializer):
    class Meta:
        model = Account
        fields = ('id', 'account_name', 'users', 'created')

NameError: name 'Account' is not defined

In [86]:
from myapp.serializers import AccountSerializer
serializer = AccountSerializer()
print(repr(serializer))
AccountSerializer():
    id = IntegerField(label='ID', read_only=True)
    name = CharField(allow_blank=True, max_length=100, required=False)
    owner = PrimarykeyRelatedField(queryset=User.objects.all())

SyntaxError: invalid syntax (<ipython-input-86-683fb34bb8ca>, line 4)

In [87]:
class AccountSerializer(serializers.ModelSerializer):
    class Meta:
        model = Account
        fields = ('id', 'account_name', 'users', 'created')

NameError: name 'Account' is not defined

In [88]:
class AccountSerializer(serializers.ModelSerializer):
    class Meta:
        model = Account
        fields = '__all__'

NameError: name 'Account' is not defined

In [89]:
class AccountSerializer(serializers.ModelSerializer):
    class Meta:
        model = Account
        exclude = ('users',)

NameError: name 'Account' is not defined

In [90]:
class AccountSerializer(serializers.Serializer):
    class Meta:
        model = Account
        fields = ('id', 'account_name', 'users', 'created')
        depth = 1

NameError: name 'Account' is not defined

In [91]:
class AccountSerializer(serializers.ModelSerializer):
    url = serializers.CharField(source='get_absolute_url', read_only=True)
    groups = serializers.PrimarykeyRelatedField(many=True)
    
    class Meta:
        model = Account

AttributeError: module 'rest_framework.serializers' has no attribute 'PrimarykeyRelatedField'

In [92]:
class AccountSerializer(serializers.ModelSerializer):
    class Meta:
        model = Account
        fields = ('id', 'account_name', 'users', 'created')
        read_only_fields = ('account_name',)

NameError: name 'Account' is not defined

In [93]:
class CreateUserSerializer(serializers.ModelSerializer):
    class Meta:
        model = User
        fields = ('email', 'username', 'password')
        extra_kwargs = {'password': {'write_only': True}}
        
    def create(self, validated_data):
        user = User(
            email=validated_data['email']
            username=validated_data['username']
        )
        user.set_password(validated_data['password'])
        user.save()
        return user

SyntaxError: invalid syntax (<ipython-input-93-74f47688e52d>, line 10)

In [94]:
class AccountSerializer(serializers.HyperlinkedModelSerializer):
    class Meta:
        model = Account
        fields = ('url', 'id', 'account_name', 'users', 'created')

NameError: name 'Account' is not defined

In [95]:
serializer = AccountSerializer(queryset, context={'request': request})

NameError: name 'AccountSerializer' is not defined

In [96]:
class AccountSerializer(serializers.HyperlinkedModelSerializer):
    class Meta:
        model = Account
        fields = ('account_url', 'account_name', 'users', 'created')
        extra_kwargs = {
            'url': {'view_name': 'accounts', 'lookup_field': 'account_name'},
            'users': {'lookup_field': 'username'}
        }

NameError: name 'Account' is not defined

In [97]:
class AccountSerializer(serializers.HyperlinkedModelSerializer):
    url = serializers.HyperlinkedIdentityField(
        view_name='accounts',
        lookup_field='slug'
    )
    users = serializers.HyperlinkedRelatedField(
        view_name='user-detail',
        lookup_field='username',
        many=True,
        read_only=True
    )
    
    class Meta:
        model = Account
        fields = ('url', 'account_name', 'users', 'created')

NameError: name 'Account' is not defined

In [98]:
class CustomListSerializer(serializers.ListSerializer):
    ...
    
class CustomSerializer(serializers.Serializer):
    ...
    class Meta:
        list_serializer_class = CustomListSerializer

In [99]:
class BookListSerializer(serializers.ListSerializer):
    def create(self, validated_data):
        books = [Book(**item) for item in validated_data]
        return Book.objects.bulk_create(books)
    
class BookSerializer(serializers.Serializer):
    ...
    class Meta:
        list_serializer_class = BookListSerializer

In [100]:
class BookListSerializer(serializers.ListSerializer):
    def update(self, instance, validated_data):
        book_mapping = {book.id: book for book in instance}
        data_mapping = {item['id']: item for item in validated_data}
        
        ret = []
        for book_id, data in data_mapping.items():
            book = book_mapping.get(book_id, None)
            if book is None:
                ret.append(self.child.create(data))
            else:
                ret.appen(self.child.update(book, data))
                
        for book_id, book in book_mapping.items():
            if book_id not in data_mapping:
                book.delete()
                
        return ret
    
class BookSerializer(serializers.Serializer):
    id = serializers.IntegerField()
    ...
    id = serializers.IntegerField(required=False)
    
    class Meta:
        list_serializer_class = BookListSerializer

In [101]:
@classmethod
    def mant_init(cls, *args, **kwargs):
        kwargs['child'] = cls()
        return CustomListSerializer(*args, **kwargs)

IndentationError: unexpected indent (<ipython-input-101-a67f7e8e9117>, line 2)

In [102]:
class HighScore(models.Model):
    created = models.DateTimeField(auto_now_add=True)
    player_name = models.CharField(max_length=10)
    score = models.IntegerField()

NameError: name 'models' is not defined

In [103]:
class HighScoreSerializer(serializers.BaseSerializer):
    def to_representation(self, obj):
        return {
            'score': obj.score,
            'player_name': obj.player_name
        }

In [104]:
@api_view(['GET'])
def high_score(request, pk):
    instance = Highscore.objects.get(pk=pk)
    serializer = HighScoreSerializer(instance)
    return Response(serializer.data)

NameError: name 'api_view' is not defined

In [105]:
@api_view(['GET'])
def all_high_scores(request):
    queryset = HighScore.objects.order_by('-score')
    serializer = HighScoreSerializer(queryset, many=True)
    return Response(serializer.data)

NameError: name 'api_view' is not defined

In [106]:
class HighScoreSerializer(serializers.BaseSerializer):
    def to_internal_value(self, data):
        score = data.get('score')
        player_name = data.get('player_name')
        
        if not score:
            raise ValidationError({
                'score': 'This field is required.'
            })
        if not player_name:
            raise ValidationError({
                'player_name': 'This field is required.'
            })
        if len(player_name) > 10:
            raise ValidationError({
                'player_name': 'May not be more than 10 characters.'
            })
            
        return {
            'score': int(score),
            'player_name': player_name
        }
    
    def to_representation(self, obj):
        return {
            'score': obj.score,
            'player_name': obj.player_name
        }
    
    def create(self, validated_data):
        return HighScore.objects.create(**validated_data)

In [107]:
class ObjectSerializer(serializers.BaseSerializer):
    def to_representation(self, obj):
        for attribute_name in dir(obj):
            attribute = getattr(obj, attribute_name)
            if attribute_name('__'):
                pass
            elif hasattr(attribute, '__call__'):
                pass
            elif isinstance(attribute, list):
                output[attribute_name] = [
                    self.to_representation(item) for item in attribute
                ]
            elif isinstance(attribute, dict):
                output[attribute_name] = {
                    str(key): self.to_representation(value)
                    for key, value in attribute.items()
                }
            else:
                output[attribute_name] = str(attribute)

In [108]:
class MyBaseSerializer(Serializer):
    my_field = serializers.CharField()
    
    def validate_my_field(self):
        ...
        
class MySerializer(MyBaseSerializer):
    ...
    

NameError: name 'Serializer' is not defined

In [109]:
class AccountSerializer(MyBaseSerializer):
    class Meta(MyBaseSerializer.Meta):
        model = Account

NameError: name 'MyBaseSerializer' is not defined

In [110]:
class MyBaseSerializer(ModelSerializer):
    my_field = serializers.CharField()
    
class MySerializer(MyBaseSerializer):
    my_field = None

NameError: name 'ModelSerializer' is not defined

In [111]:
class DynamicFieldsModelSerializer(serializers.ModelSerializer):
    def __init__(self, *args, **kwargs):
        fields = kwargs.pop('fields', None)
        
        super(DynamicFieldsModelSerializer, self).__init__(*args, **kwargs)
        
        if fields is not None:
            allowed = set(fields)
            existing = set(self.fields.keys())
            for field_name in existing - allowed:
                self.fields.pop(field_name)

In [112]:
class UserSerializer(DynamicFieldsModelSerializer):
    class Meta:
        model = User
        fields = ('id', 'username', 'email')

print UserSerializer(user)

SyntaxError: invalid syntax (<ipython-input-112-b68e144f4109>, line 6)

In [113]:
print UserSerializer(user, fields=('id', 'email'))

SyntaxError: invalid syntax (<ipython-input-113-f44d9a9c4a6a>, line 1)

In [ ]:
'''
7월 28일
'''

In [1]:
import datetime
from rest_framework import serializers
class ExampleSerializer(serializers.Serializer):
    day = serializers.DateField(initial=datetime.date.today)

In [ ]:
# Use <input type="password"> for the input.
password = serializers.CharField(
    style={'input_type': 'password'}
)

# Use a radio input instead of a select input.
color_channel = serializers.ChoiceField(
    choices=['red', 'green', 'blue'],
    style={'base_template': 'radio.html'}
)

In [2]:
serializers.DecimalField(max_digits=5, decimal_places=2)

DecimalField(decimal_places=2, max_digits=5)

In [3]:
serializers.DecimalField(max_digits=19, decimal_places=10)

DecimalField(decimal_places=10, max_digits=19)

In [4]:
DatetimeField(format=api_settings.DATETIME_FORMAT, input_formats=None)

NameError: name 'DatetimeField' is not defined

In [5]:
class CommentSerializer(serializers.ModelSerializer):
    created = serializers.DateTimeField()
    
    class Meta:
        model = Comment

NameError: name 'Comment' is not defined

In [ ]:
DateField(format=api_settings.DATE_FORMAT, input_formats=None)

In [6]:
TimeField(format=api_settings.TIME_FORMAT, input_formats=None)

NameError: name 'TimeField' is not defined

In [7]:
ChoiceField(choices)

NameError: name 'ChoiceField' is not defined

In [8]:
FileField(max_length=None, allow_empty_file=False, use_url=UPLOADED_FILES_USE_URL)

NameError: name 'FileField' is not defined

In [9]:
ImageField(max_length=None, allow_empty_file=False, use_url=UPLOADED_FILES_USE_URL)

NameError: name 'ImageField' is not defined

In [10]:
ListField(child, min_length=None, max_length=None)

NameError: name 'ListField' is not defined

In [11]:
scores = serializers.ListField(
    child=serializers.IntegerField(min_value=0, max_value=100)
)

In [12]:
class StringListField(serializers.ListField):
    child = serializers.CharField()

In [13]:
DictField(child)

NameError: name 'DictField' is not defined

In [14]:
document = DictField(child=CharField())

NameError: name 'DictField' is not defined

In [15]:
class DocumentField(DictField):
    child = CharField()

NameError: name 'DictField' is not defined

In [16]:
JSONField(binary)

NameError: name 'JSONField' is not defined

In [17]:
ReadOnlyField()

NameError: name 'ReadOnlyField' is not defined

In [18]:
class AccountSerializer(serializers.ModelSerializer):
    class Meta:
        model = Account
        fields = ('id', 'account_name', 'has_expired')

NameError: name 'Account' is not defined

In [19]:
modified = serializers.HiddenField(default=timezone.now)

In [20]:
from django.contrib.auth.models import User
from django.utils.timezone import now
from rest_framework import serializers

class UserSerializer(serializers.ModelSerializer):
    days_since_joined = serializers.SerializerMethodField()
    
    class Meta:
        model = User
        
    def get_days_since_joined(self, obj):
        return (now() - obj.date_joined).days

In [22]:
class Color(object):
    def __init__(self, red, green, blue):
        assert(red >= 0 and green >= 0 and blue >= 0)
        assert(red < 256 and green < 256 and blue < 256)
        self.red, self.green, self.blue = red, green, blue
        
class ColorField(serializers.Field):
    def to_representation(self, obj):
        return "rgb(%d, %d, %d)" % (obj.red, obj.green, obj.blue)
    
    def to_internal_value(self, data):
        data = data.strip('rgb(').rstrip(')')
        red, green, blue = [int(col) for col in data.split(',')]
        return Color(red, green, blue)